<a href="https://colab.research.google.com/github/pallak8398/sentiment-analysis-product-reviews/blob/main/Sentiment_Analysis_of_Product_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math
import warnings
warnings.filterwarnings('ignore') # Hides warning
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore",category=UserWarning)
sns.set_style("whitegrid") # Plotting style
#%matplotlib inline # Plots show up in notebook
np.random.seed(7) # seeding random number generator

csv = "1429_1.csv"
df = pd.read_csv(csv)
df.head(2)
data = df.copy()
data.describe()

,reviews.id,reviews.numHelpful,reviews.rating,reviews.userCity,reviews.userProvince
count,1.0,34131.000000,34627.000000,0.0,0.0
mean,111372787.0,0.630248,4.584573,NaN,NaN
std,NaN,13.215775,0.735653,NaN,NaN
min,111372787.0,0.000000,1.000000,NaN,NaN
25%,111372787.0,0.000000,4.000000,NaN,NaN
50%,111372787.0,0.000000,5.000000,NaN,NaN
75%,111372787.0,0.000000,5.000000,NaN,NaN
max,111372787.0,814.000000,5.000000,NaN,NaN


In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
print("Before {}".format(len(data)))
dataAfter = data.dropna(subset=["reviews.rating"]) # removes all NAN in reviews.rating
print("After {}".format(len(dataAfter)))
dataAfter["reviews.rating"] = dataAfter["reviews.rating"].astype(int)

Before 34660
After 34627


In [ ]:
split = StratifiedShuffleSplit(n_splits=5, test_size=0.2)
for train_index, test_index in split.split(dataAfter, dataAfter["reviews.rating"]): 
    strat_train = dataAfter.reindex(train_index)
    strat_test = dataAfter.reindex(test_index)

In [ ]:
len(strat_train)

27701

In [ ]:
strat_train["reviews.rating"].value_counts()/len(strat_train) # value_count() counts all the values based on column

5.0    0.685174
4.0    0.247031
3.0    0.043500
2.0    0.011696
1.0    0.011588
Name: reviews.rating, dtype: float64

In [ ]:
len(strat_test)

6926

In [ ]:
strat_test["reviews.rating"].value_counts()/len(strat_test)

5.0    0.689864
4.0    0.244730
3.0    0.042160
1.0    0.011406
2.0    0.011118
Name: reviews.rating, dtype: float64

In [ ]:
reviews = strat_train.copy()
reviews.head(2)

,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
4349,AVphgVaX1cnluZ0-DR74,"Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes...",B018Y229OU,Amazon,"Fire Tablets,Tablets,Computers & Tablets,All T...",firetablet7displaywifi8gbincludesspecialoffers...,Amazon,2015-11-28T00:00:00.000Z,2017-05-21T04:06:08Z,"2017-04-30T00:26:00.000Z,2017-06-07T08:10:00.000Z",...,True,NaN,5.0,5.0,http://reviews.bestbuy.com/3545/5025800/review...,we bought this for my 11 year old daughter and...,great for all ages,NaN,NaN,Mark
30776,AV1YE_muvKc47QAVgpwE,NaN,B00U3FPN4U,Amazon Fire Tv,"Back To College,College Electronics,College Tv...","848719057492,amazonfiretv/51454342,amazonfiret...",Amazon,2017-01-06T00:00:00.000Z,2017-09-20T05:35:55Z,"2017-08-25T22:21:42.763Z,2017-08-19T09:26:46.1...",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/4370400/review...,"I have the Roku 4, and new Apple TV, this stre...",Great streaming box,NaN,NaN,Techno


In [ ]:
corr_matrix = reviews.corr()
corr_matrix
# Here we can analyze reviews.ratings with asins

,reviews.id,reviews.numHelpful,reviews.rating,reviews.userCity,reviews.userProvince
reviews.id,NaN,NaN,NaN,NaN,NaN
reviews.numHelpful,NaN,1.00000,-0.04372,NaN,NaN
reviews.rating,NaN,-0.04372,1.00000,NaN,NaN
reviews.userCity,NaN,NaN,NaN,NaN,NaN
reviews.userProvince,NaN,NaN,NaN,NaN,NaN


Sentiment Analysis ->

In [ ]:
def sentiments(rating):
    if (rating == 5) or (rating == 4):
        return "Positive"
    elif rating == 3:
        return "Neutral"
    elif (rating == 2) or (rating == 1):
        return "Negative"
# Add sentiments to the data
strat_train["Sentiment"] = strat_train["reviews.rating"].apply(sentiments)
strat_test["Sentiment"] = strat_test["reviews.rating"].apply(sentiments)
strat_train["Sentiment"][:20]

4349     Positive
30776    Positive
28775     Neutral
1136     Positive
17803    Positive
7336     Positive
32638    Positive
13995    Positive
6728     Negative
22009    Positive
11047    Positive
22754    Positive
5578     Positive
11673    Positive
19168    Positive
14903    Positive
30843    Positive
5440     Positive
28940    Positive
31258    Positive
Name: Sentiment, dtype: object

In [ ]:
# Prepare data
X_train = strat_train["reviews.text"]
X_train_targetSentiment = strat_train["Sentiment"]
X_test = strat_test["reviews.text"]
X_test_targetSentiment = strat_test["Sentiment"]
print(len(X_train), len(X_test))
print(X_train)

27701 6926
4349     we bought this for my 11 year old daughter and...
30776    I have the Roku 4, and new Apple TV, this stre...
28775    This is my 3rd Kindle. I have continued to mov...
1136     enjoying reading via the device; lots of optio...
17803    If you want an e book reader that is light, ea...
                               ...                        
6763     great tablet. it's great for playing games and...
245      Could not have asked for a better tablet for t...
15052    Best tablet for the money Amazon delivers a go...
619      Its so perfect for my niece & she loves it. Go...
29636    Faster than the fire stick and more memory, so...
Name: reviews.text, Length: 27701, dtype: object


In [ ]:
# Replace "nan" with space
X_train = X_train.fillna(' ')
X_test = X_test.fillna(' ')
X_train_targetSentiment = X_train_targetSentiment.fillna(' ')
X_test_targetSentiment = X_test_targetSentiment.fillna(' ')

# Text preprocessing and occurance counting
from sklearn.feature_extraction.text import CountVectorizer 
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train) 
X_train_counts.shape


(27701, 12526)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer(use_idf=True)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(27701, 12526)

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
clf_linearSVC_pipe = Pipeline([("vect", CountVectorizer()), ("tfidf", TfidfTransformer()), ("clf_linearSVC", LinearSVC())])
clf_linearSVC_pipe.fit(X_train, X_train_targetSentiment)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf_linearSVC', LinearSVC())])

In [ ]:
predictedLinearSVC = clf_linearSVC_pipe.predict(X_test)
np.mean(predictedLinearSVC == X_test_targetSentiment)

0.9393589373375686

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

words_to_index = tokenizer.word_index

print(words_to_index)

{'the': 1, 'and': 2, 'to': 3, 'i': 4, 'it': 5, 'for': 6, 'a': 7, 'is': 8, 'this': 9, 'my': 10, 'great': 11, 'of': 12, 'with': 13, 'tablet': 14, 'on': 15, 'have': 16, 'you': 17, 'use': 18, 'that': 19, 'in': 20, 'but': 21, 'love': 22, 'was': 23, 'as': 24, 'easy': 25, 'so': 26, 'very': 27, 'amazon': 28, 'not': 29, 'can': 30, 'kindle': 31, 'good': 32, 'bought': 33, 'one': 34, 'fire': 35, 'price': 36, 'all': 37, 'she': 38, 'has': 39, "it's": 40, 'like': 41, 'are': 42, 'an': 43, 'we': 44, 'up': 45, 'product': 46, 'more': 47, 'be': 48, 'would': 49, 'tv': 50, 'works': 51, 'just': 52, 'or': 53, 'at': 54, 'if': 55, 'get': 56, 'echo': 57, 'had': 58, 'they': 59, 'music': 60, 'much': 61, 'alexa': 62, 'read': 63, 'kids': 64, 'apps': 65, 'loves': 66, 'than': 67, 'from': 68, 'well': 69, 'books': 70, 'reading': 71, 'do': 72, 'best': 73, 'device': 74, 'really': 75, 'me': 76, 'when': 77, 'buy': 78, 'games': 79, 'purchased': 80, 'time': 81, 'what': 82, 'her': 83, 'old': 84, 'screen': 85, 'only': 86, 'play

In [ ]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)



  return word_to_vec_map

In [ ]:
word_to_vec_map = read_glove_vector('glove.6B.50d.txt')

maxLen = 150

In [ ]:
from keras.layers import LSTM, Activation, Dropout, Dense, Input
from keras.layers.embeddings import Embedding
vocab_len = len(words_to_index)+1
embed_vector_len = word_to_vec_map['moon'].shape[0]

emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in words_to_index.items():
  embedding_vector = word_to_vec_map.get(word)
  if embedding_vector is not None:
    emb_matrix[index, :] = embedding_vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=maxLen, weights = [emb_matrix], trainable=False)


In [ ]:
from keras.models import Model
def amazon_review_rating(input_shape):

  X_indices = Input(input_shape)
  embeddings = embedding_layer(X_indices)
  X = LSTM(128, return_sequences=True)(embeddings)
  X = Dropout(0.6)(X)
  X = LSTM(128, return_sequences=True)(X)
  X = Dropout(0.6)(X)
  X = LSTM(128)(X)
  X = Dense(1, activation='sigmoid')(X)

  model = Model(inputs=X_indices, outputs=X)

  return model

In [ ]:
from keras.preprocessing.sequence import pad_sequences
X_train_indices = tokenizer.texts_to_sequences(X_train)

X_train_indices = pad_sequences(X_train_indices, maxlen=maxLen, padding='post')

In [ ]:
from tensorflow.keras.optimizers import Adam
adam = Adam(learning_rate = 0.0001)
model = amazon_review_rating((150,))
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
Y_train=[];
for x in X_train_targetSentiment:
  if x=="Positive":
    Y_train.append(1);
  elif x=="Neutral":
    Y_train.append(0);
  else: 
    Y_train.append(-1);
Y_train_numpy=np.array(Y_train);
model.fit(X_train_indices, Y_train_numpy, batch_size=64, epochs=5)

Epoch 1/5
433/433 [==============================] - 466s 1s/step - loss: 0.3356 - accuracy: 0.9291
Epoch 2/5
433/433 [==============================] - 436s 1s/step - loss: 0.2671 - accuracy: 0.9268
Epoch 3/5
433/433 [==============================] - 436s 1s/step - loss: 0.2537 - accuracy: 0.9237
Epoch 4/5
433/433 [==============================] - 433s 999ms/step - loss: 0.2489 - accuracy: 0.9229
Epoch 5/5
433/433 [==============================] - 431s 996ms/step - loss: 0.2456 - accuracy: 0.9221


In [ ]:
Y_test=[];
for x in X_train_targetSentiment:
  if x=="Positive":
    Y_test.append(1);
  elif x=="Neutral":
    Y_test.append(0);
  else: 
    Y_test.append(-1);
Y_test_numpy=np.array(Y_test);
X_test_indices = tokenizer.texts_to_sequences(X_test)


X_test_indices = pad_sequences(X_train_indices, maxlen=maxLen, padding='post')
model.evaluate(X_test_indices, Y_test_numpy)
f1_score()

866/866 [==============================] - 120s 137ms/step - loss: 0.2360 - accuracy: 0.9168


[0.23602622747421265, 0.9167900085449219]

In [ ]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D

def conv1d_model(input_shape):

  X_indices = Input(input_shape)

  embeddings = embedding_layer(X_indices)

  X = Conv1D(512,3,activation='relu')(embeddings)
  
  X = MaxPooling1D(3)(X)

  X = Conv1D(256,3,activation='relu')(X)
  
  X = MaxPooling1D(3)(X)

  X = Conv1D(256,3,activation='relu')(X)
  X = Dropout(0.8)(X)
  X = MaxPooling1D(3)(X)

  X = GlobalMaxPooling1D()(X)

  X = Dense(256, activation='relu')(X)
  X = Dense(1, activation='sigmoid')(X)

  model = Model(inputs=X_indices, outputs=X)

  return model

model_1d = conv1d_model((150,))
model_1d.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           650550    
                                                                 
 conv1d (Conv1D)             (None, 148, 512)          77312     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 49, 512)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 47, 256)           393472    
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 15, 256)          0         
 1D)                                                       

In [ ]:
model_1d.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
model_1d.fit(X_train_indices, Y_train_numpy, batch_size=64, epochs=5)

Epoch 1/5
433/433 [==============================] - 157s 360ms/step - loss: 0.2896 - accuracy: 0.9296
Epoch 2/5
433/433 [==============================] - 141s 326ms/step - loss: 0.2509 - accuracy: 0.9273
Epoch 3/5
433/433 [==============================] - 139s 321ms/step - loss: 0.2287 - accuracy: 0.9232
Epoch 4/5
433/433 [==============================] - 139s 322ms/step - loss: 0.2075 - accuracy: 0.9221
Epoch 5/5
433/433 [==============================] - 139s 321ms/step - loss: 0.1871 - accuracy: 0.9224


In [ ]:
model_1d.evaluate(X_test_indices, Y_test_numpy)

866/866 [==============================] - 38s 44ms/step - loss: 0.2171 - accuracy: 0.9236


[0.21711017191410065, 0.9236128926277161]